In [ ]:
import numpy as np
from wfdb import readsignal

In [ ]:
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampto=9999) # Default physical values
print(sig)
print(len(sig))

sig, fields=readsignal.rdsamp('sampledata/100', sampto=9999, physical=0) # Digital values
print(sig)

In [ ]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l')  # .dat, .mat... 
print(sig)
print(fields)
#gains=np.array([1,2,3])
#sig=np.subtract(sig, gains)
#print(sig)

In [ ]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31')
#print(len(sig))
#print(len(fields))
#print(fields[0])
print(sig)

# Return single numpy array
#sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', stacksegments=1)
#print(sig)
#print(sig.shape)

In [1]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)

import numpy as np
from wfdb import readsignal

sig, fields=readsignal.rdsamp('sampledata/s0010_re')
print(sig)


[[-0.2445 -0.229   0.0155 ..., -0.0015  0.06   -0.009 ]
 [-0.2425 -0.2335  0.009  ..., -0.0015  0.061  -0.01  ]
 [-0.2415 -0.2345  0.007  ..., -0.0035  0.0555 -0.0085]
 ..., 
 [ 0.152   0.2695  0.118  ...,  0.079   0.036   0.031 ]
 [ 0.136   0.256   0.1205 ...,  0.081   0.042   0.03  ]
 [ 0.135   0.2585  0.1245 ...,  0.081   0.049   0.029 ]]
